# INF8111 - Fouille de données

## TP1 Automne 2020 - Duplicate Bug Report Detection


###     Ali Arbaoui 
    



## 1 - Résumé / Overview

À cause de la complexité des systèmes informatiques, les bogues logiciels sont courants. Les entreprises, et en particulier les plus grosses, utilisent un *Bug Tracking System* (BTS), aussi appelé *Issue Tracking System*, pour organiser et suivre les rapports de bogues. En plus des développeurs et des testeurs, de nombreux projets (notamment ceux libres de droits) permettent aux utilisateurs de soumettre un rapport de bogues dans leur BTS. Pour ce faire, les utilisateurs doivent remplir un formulaire avec plusieurs champs. La majorité de ces champs contient des données catégoriques et accepte uniquement des valeurs prédéfinies (composant, version du produit et du système, etc.). Deux autres champs importants sont le résumé ou *summary* en anglais, et la description. Les utilisateurs sont libres d’écrire ce qu’ils veulent dans ces deux champs avec pour seule contrainte le nombre de caractères. La soumission de ces champs crée une page que l’on appelle rapport de bogue et qui contient toute l’information à propos du bogue.

Par manque de communication et de synchronisation, les utilisateurs ne savent pas toujours qu’un bogue a déjà été soumis et peuvent donc le soumettre à nouveau. Identifier les rapports qui correspondent au même bogue (duplicata) est une tache importante des BTSs et est le sujet de ce TP. Notre objectif est de développer un système qui comparera les nouveaux rapports de bogues avec ceux déjà soumis en fonction de leur similarité textuelle. La liste triée des rapports les plus similaires sera utilisée par un opérateur humain pour identifier manuellement si un rapport est un duplicata.

---

Due to the complexity of software systems, software bugs are prevalent. Companies, especially the larger ones, usually use a Bug Tracking System (BTS), also called Issue Tracking System, to manage and track records of bugs. Besides developers and testers, many projects, mainly open source projects, allow users to report new bugs in their BTS.
To do that, users have to fill out a form with multiple fields. An important subset of
these fields provides categorical data and only accepts values that range from a fixed list of
options (e.g. component, version and product of the system). Two other important fields
are the summary and the description. The users are free to write anything in both fields
with the only constraint that the summary has a maximum number of characters. The
submission of a form creates a page, called bug report or issue report which contains all
the information about a bug.

Due to the lack of communication and synchronization, users may not be aware that a specific bug was already submitted and may report it again. Identifying duplicate bug reports is an important task in the BTSs and is the subject of this TP. Our objective is to develop a system that will compare a new bug report with the already submitted ones and rank them based on textual similarity. This ranked list will be used by a triager to manually identify whether a report is a duplicate or not.


# 2 Installation / Setup

Pour ce TP, vous aurez besoin des librairies `numpy`, `sklearn` et `scipy` (que vous avez sans doute déjà), ainsi que la librairie `nltk`, qui est une libraire utilisée pour faire du traitement du language (Natural Language Processing, NLP). Installez les libraires en question et exécutez le code ci-dessous :

---

For this assignment, you need the `numpy`, `sklearn` and `scipy` libraries (which you may already have), as well as the `nltk` library, which is used for Natural Language Processing (NLP). Please run the code below to install the packages needed for this assignment.

In [1]:
pip install --user numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --user sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --user scipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --user nltk 

Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk

In [6]:
import json

In [7]:
pip install --user tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install jupyterthemes

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 3 - Jeu de données / Data

Téléchargez l'archive à l'adresse suivante: https://drive.google.com/file/d/14BrFaOiVDIcpjwpm5iAkAy0v_moZyGiP/view?usp=sharing

In this zip file, there are: 

L'archive contient:
1. test.json: Ce fichier contient les duplicate bug reports that will be used to evaluate our system.
2. threads: Ce dossier contient le code HTML des bug report. Chaque fichier HTML est nommé selon le motif **bug_report_id.html**.


L'image ci-dessous illustre un exemple de bug report:

![https://ibb.co/tqyRM4L](bug_report.png)

- A : identifiant du bug report
- B : date de création
- C : résumé
- D : composant
- E : produit
- F : l'identifiant du rapport dont le bug report est dupliqué
- G : description


Le script suivant charge le jeu de données de test et définit certaines variables globales:

---

Please download the zip file at the following adress: https://drive.google.com/file/d/14BrFaOiVDIcpjwpm5iAkAy0v_moZyGiP/view?usp=sharing

This archive contains: 

1. test.json: This file contains duplicate bug reports that will be used to evaluate our system.
2. bug_reports: It is a folder that contains the bug report html source. Each html file name follows the pattern **bug_report_id.html**.


Figure below depicts an bug report page example:

![https://ibb.co/tqyRM4L](bug_report.png)


- A : bug report id
- B : creation date
- C : summary
- D : component
- E : product
- F : the report id which the bug report is duplicate
- G : description

 The following script loads the test dataset and define some global variables:

In [9]:
import os

# define the folder path that contain the data
# FOLDER_PATH = "Define folder path that contain threads folder and test.json"
FOLDER_PATH = "dataset/"
PAGE_FOLDER = os.path.join(FOLDER_PATH, 'bug_reports')


# Load the evaluation dataset
import json
os.path.exists("test.json")

test = json.load(open(os.path.join(FOLDER_PATH, "test.json")))

# 4 - Web scraping

"Le *web scraping* (parfois appelé harvesting) est une technique d'extraction du contenu de sites Web, via un script ou un programme, dans le but de le transformer pour permettre son utilisation dans un autre contexte, par exemple le référencement." [Wikipedia](https://fr.wikipedia.org/wiki/Web_scraping)

---

*Web scraping* also called harvesting consists in extracting relevant data from web pages and prepare it for computational analysis.


In [10]:
pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import lxml.html

In [12]:
from bs4 import BeautifulSoup

## 4.1 - Question 1 (2 points)

Implémentez *extract_data_from_page*. Cette fonction extrait l'information suivante du code HTML : identifiant du rapport de bogue, date de création, titre, produit, composant, identifiant du rapport de bogue bug dont il est un duplicata, résumé et description. 


La fonction *extract_data_from_page* retourne un dictionnaire avec la structure suivante :
```
 {"report_id": int, 
  "dup_id": int or None (the report id which it is duplicate), 
  "component": string, 
  "product": string, 
  "summary": str, 
  "description": string, 
  "creation_date": int} 
```

Par exemple, quand la fonction *extract_data_from_page* reçoit le rapport "bug_report/7526.html", elle retourne :

```
{
"report_id": 7526,
"dup_id": 7799,
"product": "core graveyard",
"component":  tracking,
"summary": "Apprunner crashes on exit",
"description": "Apprunner crashes on exit, without fail. The browser window closes, but the
console window hangs around. I end up having to kill winoldap with the \\"Close
Program\\" dialog (Ctrl-Alt-Del).",
"creation_date": 928396140
}
```

**La date de création doit être représentée comme un timestamp (entier). Si un bug n'est pas un duplicata, alors dup_id doit être None.**

*Indice: lxml parse est plus rapide que html.parser*

---

Implement *extract_data_from_page*. This function extracts the following information from the html: bug report id, creation date, title, product, component, the report id which it is duplicate, summary and description.

The *extract_data_from_page* function returns a dictionary with the following structure:
```
 {"report_id": int, 
  "dup_id": int or None (the report id which it is duplicate), 
  "component": string, 
  "product": string, 
  "summary": str, 
  "description": string, 
  "creation_date": int} 
```

For instance, when extract_data_from_page receives "bug_report/7526.html", it returns:

```
{
"report_id": 7526,
"dup_id": 7799,
"product": "core graveyard",
"component":  tracking,
"summary": "Apprunner crashes on exit",
"description": "Apprunner crashes on exit, without fail. The browser window closes, but the
console window hangs around. I end up having to kill winoldap with the \\"Close
Program\\" dialog (Ctrl-Alt-Del).",
"creation_date": 928396140
}
```

**Creation date have to be represented as timestamp. If bug report is not duplicate, dup_id have to be None.**

*HINT: lxml parse is faster than html.parser*


In [13]:
def extract_data_from_page(pagepath):
    """
    Scrap bug report content from bug report html.
    
    :param pagepath: the path of html file.
    :return: 
        {
        "report_id": int,
        "dup_id": int or None (the report id which it is duplicate), 
        "component": string, 
        "product": string, 
        "summary": str, 
        "description": string, 
        "creation_date": int
        }
    """
    with open(pagepath,'r', encoding="utf8") as html_file:
        content = html_file.read()
        
    bug_scrape = BeautifulSoup(content,'lxml')
    
    # Scraping parameters
    report_id = int(bug_scrape.find('span', id = 'field-value-bug_id').text.split()[1])
    creation_date = int(bug_scrape.find('span', class_ = 'rel-time').get('data-time'))
    product = bug_scrape.find('span', id ='product-name').text.split()[0] # component and product texts contain whitespaces and new lines that's why we use split
    component = bug_scrape.find('span', id = 'component-name').text.split()[0]
    summary = bug_scrape.find('h1', id = 'field-value-short_desc').text
    description = bug_scrape.find('pre', id = 'ct-0').text
    
    # dup_id
    if 'duplicate' in bug_scrape.find('span', id = 'field-value-status-view').text.lower():
        dup_id = int(bug_scrape.find('span', id = 'field-value-status-view').a.text.split()[1])
    else:
        dup_id = None
    
    return {
        "report_id": report_id,
        "dup_id": dup_id, 
        "component": component, 
        "product": product, 
        "summary": summary, 
        "description": description, 
        "creation_date": creation_date
        }

Here we are gonna test on some buggs to see if our fucntions works correctly.

In [14]:
%%time
print(extract_data_from_page('dataset/bug_reports/63.html'))

{'report_id': 63, 'dup_id': None, 'component': 'Aurora/RDF', 'product': 'MozillaClassic', 'summary': 'NavCenter closing up garbage-resource-string', 'description': 'Created by Jukka Santala (donwulff@iki.fi) on Tuesday, April 7, 1998 5:13:38 AM PDT\nAdditional Details :\nSometimes (don\nUpdated by Mike Pinkerton (pinkerton@netscape.com) on Tuesday, April 7, 1998 8:40:37 AM PDT\nAdditional Details :\nThis is probably cross platform. I have seen that on my mac builds as well.\nChanging platform/OS to "ALL".\nUpdated by Jukka Santala (donwulff@iki.fi) on Wednesday, April 8, 1998 6:00:55 PM PDT\nAdditional Details :\nFor some reason, adding the previous comment ate most of the original problem\ndescription; consider that a bug report for the Bugzilla code. In short though,\n/modules/rdf/src/nlcstore.c has at position 210 rows into itself code, where\nduring Navigator shutdown resourceID(u) (which expands to ->url macro) sometimes\nreturns "garbage". At first I thought it was just a NULL po

## 4.3 - Extraire le texte du code HTML / Extract text from HTML


In [15]:
import os
from multiprocessing import Pool, TimeoutError
from time import time
import json
import tqdm
from tqdm import tqdm

# Index each thread by its id
index_path = os.path.join(PAGE_FOLDER, 'bug_reports.json').replace('\\','/')


if os.path.isfile(index_path):
    # Load threads that webpage content were already extracted.
    report_index = json.load(open(index_path))
    
else:
    # Extract webpage content

    # This can be slow (around 10 minutes). Test your code with a small sample. lxml parse is faster than html.parser
    files = [os.path.join(PAGE_FOLDER, filename).replace('\\','/') for filename in os.listdir(PAGE_FOLDER)]
    reports = [extract_data_from_page(f) for f in tqdm(files)]
    report_index = dict(((report['report_id'], report) for report in reports ))

    # Save preprocessed bug reports
    json.dump(report_index, open(index_path,'w'))
    

100%|██████████████████████████████████████████████████████████████████████████████| 9998/9998 [10:10<00:00, 16.38it/s]


# 5 - Prétraitement des données / Data Preprocessing

Le prétraitement des données est une tache cruciale en fouille de données. Cette étape nettoie et transforme les données brutes dans un format qui permet leur analyse, et leur utilisation avec des algorithmes de *machine learning*. En traitement des langages (natural language processing, NLP), la *tokenization* et le *stemming* sont des étapes cruciales. De plus, vous implémenterez une étape supplémentaire pour filtrer les mots sans importance.

---

Preprocessing is a crucial task in data mining. The objective is to clean and transform the raw data in a format that is better suited for data analysis and machine learning techniques. In natural language processing (NLP), *tokenization* and *stemming* are two well-known preprocessing steps. Besides these two steps, we will implement an additional step that is designed exclusively for the twitter domain.

## 5.1 - Tokenization

Cette étape permet de séparer un texte en séquence de *tokens* (= jetons, ici des mots, symboles ou ponctuation). Par example, la phrase *"It's the student's notebook."* peut être séparé en liste de tokens de cette manière: ["it", " 's", "the", "student", " 's", "notebook", "."].

---

In this preprocessing step, a *tokenizer* is responsible for breaking a text in a sequence of tokens (words, symbols, and punctuation). For instance, the sentence *"It's the student's notebook."* can be split into the following list of tokens: ['It', "'s", 'the', 'student', "'s", 'notebook', '.'].


### 5.1.1 - Question 2 (0.5 points) 
Implémentez la fonction suivante :

- **tokenize_space** qui tokenize le texte à partir des blancs (espace, tabulation, nouvelle ligne). Ce tokenizer est naïf.
- **tokenize_nltk** qui utilise le tokenizer par défaut de la librairie nltk (https://www.nltk.org/api/nltk.html).
- **tokenize_space_punk** replace la ponctuation par des espaces puis tokenize les tokens qui sont séparés par des blancs (espace, tabulation, retour à la ligne).

**Tous les tokenizers doivent mettre les tokens en minuscule.**

---

Implement the following functions: 
- **tokenize_space** tokenizes the tokens that are separated by whitespace (space, tab, newline). This is a naive tokenizer.
- **tokenize_nltk** uses the default method of the nltk package (https://www.nltk.org/api/nltk.html) to tokenize the text.
- **tokenize_space_punk** replaces the punctuation to space and then tokenizes the tokens that are separated by whitespace (space, tab, newline).

**All tokenizers have to lowercase the tokens.**


In [16]:
import nltk
from nltk.tokenize import word_tokenize

In [17]:
import string

In [18]:
def tokenize_space(text):
    """
    Tokenize the tokens that are separated by whitespace (space, tab, newline). 
    We consider that any tokenization was applied in the text when we use this tokenizer.
    
    For example: "hello\tworld of\nNLP" is split in ['hello', 'world', 'of', 'NLP']
    """
    # return a list of tokens
    
    return text.lower().split()
        
def tokenize_nltk(text):
    """
    This tokenizer uses the default function of nltk package (https://www.nltk.org/api/nltk.html) to tokenize the text.
    """
    # return a list of tokens
    
    return word_tokenize(text.lower())


def tokenize_space_punk(text):
    """
    This tokenizer replaces punctuation to spaces and then tokenizes the tokens that are separated by whitespace (space, tab, newline).
    """
    for punctuation in string.punctuation:
        text = text.replace(punctuation,' ')
    
    return text.lower().split()
    
        

In [19]:
%%time
# To test the different tokenizers above

text_example = "hello\tworld of\nNLP"
print(tokenize_space(text_example)==['hello', 'world', 'of', 'nlp'])
print(tokenize_nltk(text_example)==['hello', 'world', 'of', 'nlp'])
print(tokenize_space_punk(text_example)==['hello', 'world', 'of', 'nlp'])

print('\n')

text_example2 = "hello\tworld of\nNLP,Funky !Stuff... really NICE.. TO SEE"
print("Using the tokenize_space tokenizer :",tokenize_space(text_example2))
print("Using the tokenize_nltk tokenizer :",tokenize_nltk(text_example2))
print("Using the tokenize_space_punk tokenizer :",tokenize_space_punk(text_example2))

True
True
True


Using the tokenize_space tokenizer : ['hello', 'world', 'of', 'nlp,funky', '!stuff...', 'really', 'nice..', 'to', 'see']
Using the tokenize_nltk tokenizer : ['hello', 'world', 'of', 'nlp', ',', 'funky', '!', 'stuff', '...', 'really', 'nice', '..', 'to', 'see']
Using the tokenize_space_punk tokenizer : ['hello', 'world', 'of', 'nlp', 'funky', 'stuff', 'really', 'nice', 'to', 'see']
CPU times: total: 31.2 ms
Wall time: 29.9 ms


## 5.2 - Suppression des mots vides / Stop words removal

### 5.2.1 - Question 3 (0.5 point)

Certains tokens sont sans importance pour la comparaison, car ils apparaissent dans la majorité des discussions. Les supprimer réduit la dimension du vecteur et accélère les calculs.

Expliquez quels tokens sont sans importances pour la comparaison des discussions. Implémentez la fonction filter_tokens qui retire ces mots de la liste des tokens.

---

There are a set of tokens that are not significant to the similarity comparison since they appear in most of bug report pages. Thus, removing them decreases the vector dimensionality and turns the similarity calculation computationally cheaper.

Describe the tokens that can be removed without affecting the similarity comparison? Moreover, implement the function filter_tokens that removes these words from a list of tokens.


In [20]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [21]:
print(stop_words)

{'not', 'their', 'is', 'shouldn', "aren't", 'about', 'then', 'wasn', 'mustn', 'couldn', 've', 'once', 'am', 'where', 'those', 'out', 'other', 'all', "you'd", 'doesn', 'did', 'be', "won't", 'yours', 'was', 'a', 'does', 'on', 'and', 'him', 'y', 'ours', 'when', 'more', 'whom', "haven't", 'above', 'up', 'been', 'm', "couldn't", "you're", 'off', 'being', 'during', 'at', 'such', 'himself', "that'll", "hasn't", 'my', 'further', 'doing', "you'll", 'nor', 'this', 'itself', 't', 'i', 'haven', 'its', "shouldn't", 'our', 'any', "wouldn't", 'as', 'of', 'can', 'these', 'do', 'against', 'same', 'the', 'having', "doesn't", 'isn', 'ma', 'myself', 'they', 'while', 'mightn', 'won', 'how', "mightn't", 'from', 'in', 'weren', 'for', 'themselves', 'she', 'who', "should've", 'over', 's', 'below', 'few', 're', 'his', 'very', 'you', 'again', "shan't", 'don', "isn't", "mustn't", 'if', 'd', 'each', 'no', 'there', 'o', 'hasn', 'own', "don't", 'me', 'under', 'we', 'shan', 'had', 'needn', 'but', 'ain', 'through', 't

In [22]:
def filter_tokens(tokens):
    filtered_tokens = []
    for token in tokens:
        if token not in stop_words:
            filtered_tokens.append(token)
    return filtered_tokens

## 5.3 - Racinisation / Stemming

La racinisation, ou *stemming* en anglais, est un procédé de transformation des flexions en leur radical ou racine. Par exemple, en anglais, la racinisation de "fishing", "fished" and "fish" donne "fish" (stem). 

---

The process to convert tokens with the same stem (word reduction that keeps word prefixes) to a standard form is called *stemming*. For instance, the word "fishing", "fished" , "fish" and "fisher" are reduced to the stem "fish".



In [23]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')


word1 = ['I', 'tried', 'different', 'fishes']

print([ stemmer.stem(w) for w in word1])

word2 = ['I', 'will', 'tries', 'only', 'one', 'fishing']
print([ stemmer.stem(w) for w in word2])

['i', 'tri', 'differ', 'fish']
['i', 'will', 'tri', 'onli', 'one', 'fish']


### 5.3.1 - Question 4 (0.5 point) 

Expliquez comment et pourquoi le stemming est utile à notre système.

---

Explain how stemming can benefit our system?

We can notice that bug reporters when summarising or describing the bugs reported, they often reports similar expressions using different forms of some of the keywords of their report, so there might be a same meaning in the words of two different reporters but it won't count in the similarity calculation since it's not the same token. For example, the Bug 7526 is a duplicate of the bug 7799, the summary of the first one is "Apprunner crashes on exit" whereas the summary of the original bug is "Crash on exit bugs", so here without enabling stemming, our tokens set would contain "crash" and "crashes" where these two have the same meaning, so with stemming we expect to have more similarity between some of the documents when reducing some of the influential key words to their standard forms.

# 6 - Représentation des données / Data Representation

# 6.1 - Bag of Words

De nombreux algorithmes demandent des entrées qui soient toutes de la même taille, ce qui n'est forcément le cas pour des types de données comme les textes, qui peuvent avoir un nombre variable de mots.  

Par exemple, considérons la phrase 1, ”Board games are much better than video games” et la phrase 2, ”Monopoly is an awesome game!”. La table ci-dessous montre un exemple d'un moyen de représentation de ces deux phrases en utilisant une représentation fixe : 

|<i></i>     | an | are | ! | Monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 1 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Chaque colonne représente un mot du vocabulaire (de longueur 13), tandis que chaque ligne contient l'occurrence des mots dans une phrase. Ainsi, la valeur 2 à la position (1,7) est due au fait que le mot *"games"* apparaît deux fois dans la phrase 1. 

Ainsi, chaque ligne étant de longueur 13, on peut les utiliser comme vecteur pour représenter les phrases 1 et 2. Ainsi, c'est cette méthode que l'on appelle *Bag-of-Words* : c'est une représentation de documents par des vecteurs dont la dimension est égale à la taille du vocabulaire, et qui est construite en comptant le nombre d'occurrences de chaque mot. Ainsi, chaque token est ici associé à une dimension.

---

Many algorithms only accept inputs that have the same size. However, there are some data types whose sizes are not fixed, for instance, a text can have an unlimited number of words. Imagine that we retrieve two tweets: ”Board games are much better than video games” and ”Monopoly is an awesome game!”. These sentences are respectively named as Sentences 1 and 2. The table below depicts how we could represent both sentences using a fixed representation.

|            | an | are | ! | monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 0 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Each column of this table 2.1 represents one of 13 vocabulary words, whereas the rows contains the word
frequencies in each sentence. For instance, the cell in row 1 and column 7 has the value 2
because the word games occurs twice in Sentence 1. Since the rows have always 13 values, we
could use those vectors to represent the sentences 1 and 2. The table above illustrates a technique called bag-of-words. Bag-of-words represents a document as a vector whose dimensions are equal to the number of times that vocabulary words appeared in the document. Thus, each token will be related to a dimension, i.e. an integer.

<!-- Using raw frequency in the bag-of-words can be problematic. The word frequency distribution
is skewed - only a few words have high frequencies in a document. Consequently, the
weight of these words will be much bigger than the other ones which can give them more
impact on some tasks, like similarity comparison. Besides that, a set of words (including
those with high frequency) appears in most of the documents and, therefore, they do not
help to discriminate documents. For instance, the word *of* appears in a significant
number of tweets. Thus, having the word *of* does not make
documents more or less similar. However, the word *terrible* is rarer and documents that
have this word are more likely to be negative. TF-IDF is a technique that overcomes the word frequency disadvantages. -->

### 6.1.2 - Question 5 (2 points)


Implémentez le Bag-of-Words en pondérant le vecteur par la fréquence de chaque mot.

**Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html) de scipy.**

---

Implement the bag-of-words model that weights the vector with the absolute word frequency.

**For this exercise, you cannot use any external python library (e.g. scikit-learn). However, if you have a problem with memory size, you can use the scipy class [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).**



In [24]:
from scipy.sparse import csr_matrix

In [25]:
def transform_count_bow(X):
    """
    This method preprocesses the data using the pipeline object, relates each token to a specific integer and  
    transforms the text in a vector. Vectors are weighted using the token frequencies in the sentence.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document
    """ 
    set_tokens = set() # It will contain the tokens we're dealing with
    
    for sentence in X:
        set_tokens.update(sentence)
        
    tokens_dict = {val: id for id,val in enumerate(set_tokens)}  # Here we give an order of the tokens  
    
    data, indices, indptr = [], [], [0]
    
    for sentence in X:   
        
        for word in set(sentence):
            
            data.append(sentence.count(word))
            indices.append(tokens_dict[word])
        
        indptr.append(indptr[-1]+len(set(sentence)))
            
    return csr_matrix((data, indices, indptr), shape=(len(X),len(set_tokens)))
        

In [26]:
%%time
tokens_example = [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]
count_bow_mat = transform_count_bow(tokens_example)
print(count_bow_mat.toarray())

[[0 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
 [0 0 1 1 1 0 1 0 0 1 1 0 1 0 1 0]]
CPU times: total: 0 ns
Wall time: 6.98 ms


In [27]:
reports1 = [r for r in report_index.values()]
reports1

[{'report_id': 1000,
  'dup_id': None,
  'component': 'Layout',
  'product': 'MozillaClassic',
  'summary': 'Core Dump in laysel.c',
  'description': "Goto the above url and click on the talkback graphics...BOOM...\n\nCore Dump in laysel.c\n\nHere is a stack trace,  The problem is the begin is NULL and being\ndereferenced.  There was no selection when i went to the above url.\n\n\n#0  0x403f891d in lo_HitLine (context=0x861fe00, state=0x862de00, x=393, y=15,\n    requireCaret=1 '\\001', result=0xbfffd2c4) at laysel.c:4487\n---Type <return> to continue, or q <return> to quit---\n4487            if ( begin->type != LO_LINEFEED && end->type == LO_LINEFEED ) {\n\n(gdb) bt\n#0  0x403f891d in lo_HitLine (context=0x861fe00, state=0x862de00, x=393, y=15,\n    requireCaret=1 '\\001', result=0xbfffd2c4) at laysel.c:4487\n#1  0x403f9236 in LO_Hit (context=0x861fe00, x=393, y=15,\n    requireCaret=1 '\\001', result=0xbfffd2c4, layer=0x8668b00) at laysel.c:4941\n#2  0x403f93de in LO_Click (context=

## 6.2 - TF-IDF

L'utilisation de la fréquence d'apparition brute des mots, comme c'est le cas avec le bag-of-words, peut être problématique. En effet, peu de tokens auront une fréquence très élevée dans un document, et de ce fait, le poids de ces mots sera beaucoup plus grand que les autres, ce qui aura tendance à biaiser l'ensemble des poids. De plus, les mots qui apparaissent dans la plupart des documents n'aident pas à les discriminer. Par exemple, le mot "*de*" apparaît dans beaucoup de documents de la base de données, et pour autant, avoir ce mot en commun ne permet pas de conclure que des documents sont similaires. Au contraire, le mot "*génial*" est plus rare, mais les documents qui contiennent ce mot sont plus susceptibles d'être positifs. TF-IDF est donc une méthode qui permet de pallier à ce problème.

TF-IDF pondère le vecteur en utilisant une fréquence de document inverse (IDF) et une fréquence de termes (TF).

TF est l'information locale sur l'importance qu'a un mot dans un document donné, tandis que IDF mesure la capacité de discrimination des mots dans un jeu de données. 

L'IDF d'un mot se calcule de la façon suivante:

\begin{equation}
  \text{idf}_i = \log\left( \frac{N}{\text{df}_i} \right),
\end{equation}

Avec $N$ le nombre de documents dans la base de données, et $\text{df}_i$ le nombre de documents qui contiennent le mot $i$.

Le nouveau poids $w_{ij}$ d'un mot $i$ dans un document $j$ peut ensuite être calculé de la façon suivante:

\begin{equation}
  w_{ij} = \text{tf}_{ij} \times \text{idf}_i,
\end{equation}

avec $\text{tf}_{ij}$ la fréquence du mot $i$ dans le document $j$

---

Using raw frequency in the bag-of-words can be problematic. The word frequency distribution
is skewed - only a few words have high frequencies in a document. Consequently, the
weight of these words will be much bigger than the other ones which can give them more
impact on some tasks, like similarity comparison. Besides that, a set of words (including
those with high frequency) appears in most of the documents and, therefore, they do not
help to discriminate documents. For instance, the word *of* appears in a significant
number of tweets. Thus, having the word *of* does not make
documents more or less similar. However, the word *aeroplane* is rarer and documents that
have this word are more likely to be similar. TF-IDF is a technique that overcomes the word frequency disadvantages.

TF-IDF weights the vector using inverse document frequency (IDF) and word frequency, called term frequency (TF).
TF is the local information about how important is a word to a specific document.  IDF measures the discrimination level of the words in a dataset.  Common words in a domain are not helpful to discriminate documents since most of them contain these terms. So, to reduce their relevance in the documents, these words should have low weights in the vectors. 
The following equation computes the word IDF:
\begin{equation}
  idf_i = \log\left( \frac{N}{df_i} \right),
\end{equation}
Where $N$ is the number of documents in the dataset $df_i$ is the number of documents that contain a word $i$.
The new weight $w_{ij}$ of a word $i$ in a document $j$ using TF-IDF is computed as:
\begin{equation}
  w_{ij} = tf_{ij} \times idf_i,
\end{equation}
Where $tf_{ij}$ is the term frequency of words $i$ in the document $j$.



### 6.2.1 - Question 6 (2.5 points)

Implémentez le bag-of-words avec la pondération de TF-IDF

**Pour cette question, vous ne pouvez pas utiliser de librairie Python externe comme scikit-learn, hormis si vous avez des problèmes de mémoire, vous pouvez utiliser la classe [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html) de scipy.**

---

Implement a bag-of-words model that weights the vector using TF-IDF.

**For this exercise, you cannot use any external python library (e.g. scikit-learn). However, if you have a problem with memory size, you can use the scipy class [sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html)**



In [28]:
import numpy as np

In [29]:
def transform_tf_idf_bow(X):
    """
    This method preprocesses the data using the pipeline object, calculates the IDF and TF and 
    transforms the text in vectors. Vectors are weighted using TF-IDF method.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document
    """
    set_tokens = set() # It will contain the tokens we're dealing with
    
    for sentence in X:
        set_tokens.update(sentence)
        
    tokens_dict = {val: i for i,val in enumerate(set_tokens)}  # Here we give an order of the tokens  

    token_count, indices, indptr = [], [], [0]
    
    dict_document_count = dict((val,0) for val,i in tokens_dict.items())
    
    for sentence in X:   
        
        for word in set(sentence):
            
            token_count.append(sentence.count(word))
            indices.append(tokens_dict[word])
            dict_document_count[word] += 1             
        
        indptr.append(indptr[-1]+len(set(sentence)))
    
    idf = dict((i, np.log(len(X)/dict_document_count[val])) for val, i in tokens_dict.items())
    data = [token_count[i]*idf[indices[i]] for i in range(len(indices))]   
    

    return csr_matrix((data, indices, indptr), shape=(len(X),len(set_tokens)))

In [30]:
%%time
tokens_example = [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]
tfidf_matrice= transform_tf_idf_bow(tokens_example)
print(tfidf_matrice.toarray())

[[0.         1.09861229 0.         0.         0.         1.09861229
  0.         0.         1.09861229 0.         0.         1.09861229
  0.         1.09861229 0.         0.        ]
 [1.09861229 0.         0.         0.         0.         0.
  0.         1.09861229 0.         0.         0.         0.
  0.         0.         0.         1.09861229]
 [0.         0.         1.09861229 1.09861229 1.09861229 0.
  1.09861229 0.         0.         1.09861229 1.09861229 0.
  1.09861229 0.         1.09861229 0.        ]]
CPU times: total: 15.6 ms
Wall time: 1.99 ms


In [31]:
%%time
transform_tf_idf_bow([['A', 'B','C','D','D','D'],['A','A','A'],['A','A'],['A','B','B'],['B']]).toarray()

CPU times: total: 0 ns
Wall time: 0 ns


array([[1.60943791, 0.51082562, 0.22314355, 4.82831374],
       [0.        , 0.        , 0.66943065, 0.        ],
       [0.        , 0.        , 0.4462871 , 0.        ],
       [0.        , 1.02165125, 0.22314355, 0.        ],
       [0.        , 0.51082562, 0.        , 0.        ]])

# 7 - Notre système / Our System

## 7.1 - Question 7 (1 point)

La *pipeline* est la séquence d'étapes de prétraitement des données qui transforme les données brutes dans un format qui permet leur analyse. Pour notre problème, implémentez un pipeline composé des étapes suivantes :

1. Concatène le texte du résumé et description
2. Tokenize le texte, retire les stop words et stem le tokens. 
3. Génère la représentation vectorielle avec transform_tf_idf_bow ou transform_count_bow.
4. Encode données catégorielles (produit et composant) en entier 

La pipeline (fonction nlp_pipeline) prend en entrée la liste des rapports de bogues (liste des dictionnaires qui contiennent les informations des rapports), le type de tokenization, le type de vectorizer, un booléen qui active ou désactive la suppression des tokens inutiles et un booléen qui active ou désactive le stemming. La fonction nlp_pipeline retourne un tuple ($p$, $c$, $M$) :
- $p$ est le vecteur qui contient l'identifiant des produits des rapports de bogues
- $c$ est le vecteur qui contient l'identifiant des composants des rapports de bogues
- $M$ est une matrice avec la représentation du texte.

Le i-ème élément de $p$, $c$ et $M$ sont le produit, composant et la représentation du texte du i-ème rapport de bogue.


---

The pipeline is a sequence of preprocessing steps that transform the raw data to a format that is suitable for your problem. For our problem, you have to implement a pipeline that:

1. Concatenate the summary and description
2. Perform the tokenization, stop word removal and stemming in textual data
3. Generate the vector representation using transform_tf_idf_bow or transform_count_bow
4. Encode the categorical data (the component and product) to integers


The pipeline (nlp_pipeline function) receives a list of bug reports (dictionary that contains the report information), tokenization type, vectorizer type, a flag that enables or disable the insignificant token removal and a flag that turn stemming on or off. The nlp_pipeline function returns a tuple ($p$, $c$, $M$):
- $p$ is a vector that contains the product values of the bug reports
- $c$ is a vector that contains the component values of the bug reports
- $M$ is a matrix with the text representation.

The $i$-th element of $p$, $c$ and $M$ are the product, component and text representation of the $i$-th report in bug_reports, respectively.



In [32]:
def nlp_pipeline(bug_reports, tokenization_type, vectorizer_type, enable_stop_words, enable_stemming):
    """
    Preprocess and vectorize the threads.
    
    bug_reports: list of all bug reports([dict()]).
    tokenization_type: two possible values "space_tokenization" and "nltk_tokenization".
                            - space_tokenization: tokenize_space function is used to tokenize.
                            - nltk_tokenization: tokenize_nltk function is used to tokenize.
                            - space_punk_tokenization: tokenize_space_punk is used to tokenize.
                            
    vectorizer_type: two possible values "count" and "tf_idf".
                            - count: use transform_count_bow to vectorize the text
                            - tf_idf: use transform_tf_idf_bow to vectorize the text
                            
    enable_stop_words: Enables or disables the insignificant stop words removal
    enable_stemming: Enables or disables steming
    
    return: tuple ($p$, $c$, $M$)
    """
    
    # Concatenatig summary and description
    summary_desc = [' '.join([report['summary'],report['description']]) for report in bug_reports]
    
    # Set of all the bug reports components and a set of all the bug reports products
    components_set = set(report['component'] for  report in bug_reports)
    products_set = set(report['product'] for report in bug_reports)
    
    # Tokenizing the new text
    reports_cleaning = [tokenization_type(report_text) for report_text in summary_desc]
           
    if enable_stop_words: # Removing the stop words when the parameter takes True
        reports_cleaning = [filter_tokens(tokens) for tokens in reports_cleaning]
    
    if enable_stemming: # Stemming when getting True for this parameter
        reports_cleaning = [[stemmer.stem(word) for word in report] for report in reports_cleaning]  
    
    vectorized_reports = vectorizer_type(reports_cleaning) # The matrix with the vectors representation
    components_dict = {val: id for id,val in enumerate(components_set)} # Encoding components to integer
    products_dict = {val : id for id,val in enumerate(products_set)} # Encoding products to integer
    
    components_vals = [components_dict[report['component']] for report in bug_reports] # Vector containing components values
    products_vals = [products_dict[report['product']] for report in bug_reports] # Vector containing products values
    
    return products_vals, components_vals, vectorized_reports


## 7.2 - Question 8 (1 point)

Implémentez la fonction rank qui retourne la liste des indices des rapports de bogues triée en fonction de la similarité entre les rapports de bug (candidat) et du nouveau rapport (requête). Vous utiliserez la fonction de similarité suivante pour comparer deux rapports :

$$
\mathrm{SIM}(q,r) = w_1 * F_1(q,r) + w_2 * F_c(q,r) + w_3 * cos\_sim(\mathrm{txt}_q, \mathrm{txt}_c),
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{si } p_q= p_r\\
    0,              & \text{autrement},
\end{cases}
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{si } c_q = c_r\\
    0,              & \text{autrement},
\end{cases}
$$

Où $c_q$ et $c_r$ sont les composants de la requête et du candidat,
 $p_q$ et $p_r$ sont les produits de la requête et du candidat,
 $\mathrm{txt}_q$ et $\mathrm{txt}_c$ sont les représentations du texte de la requête et du candida. Les paramètres 
 w_1, w_2 et w_3 doivent être réglés.
 

**Pour cette question, la requête doit être retirée de la liste triée (sortie de la fonction rank).**

*Pour de meilleures performances, vous pouvez utiliser la fonction cos d'une libraire (par exemple [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)) pour calculer la similarité. Il est préférable de faire des opérations matricielles.*

---

Implement the function rank that returns a list of reports indexes sorted by similarity of the bug reports (candidates) and new bug report (query). We use the following similarity function to compare two bug reports:

$$
\mathrm{SIM}(q,r) = w_1 * F_1(q,r) + w_2 * F_c(q,r) + w_3 * cos\_sim(\mathrm{txt}_q, \mathrm{txt}_c),
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{if } p_q= p_r\\
    0,              & \text{otherwise},
\end{cases}
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{if } c_q = c_r\\
    0,              & \text{otherwise},
\end{cases}
$$
Where $c_q$ and $c_r$ are the query and candidate components,
 $p_q$ and $p_r$ are the query and candidate products,
 $\mathrm{txt}_q$ and $\mathrm{txt}_c$ are the query and candidate textual representations, respectively. The parameters 
 w_1, w_2 and w_3 are to be tuned.
 

**In this question, the query has to  be removed in the sorted list (rank output).**

*For better performance, you can use the cosine similarity from a library (e.g. [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)). Also, we recommend performing matrix operations.*

In [33]:
%%time
pro, comp, K = nlp_pipeline(reports1, tokenize_space, transform_tf_idf_bow, False, False)

CPU times: total: 5.22 s
Wall time: 5.7 s


In [34]:
# Cosine Similarity from scratch

def cos_sim(A,B):
    prod = np.dot(A, B.transpose()) # prod[i,j] equals the scalar product between the ith row of A and jth row of B
    dist_A = np.sqrt(np.sum(np.square(A), axis=1)) # dist_A[i] is the distance of the ith row of A
    dist_B = np.sqrt(np.sum(np.square(A), axis=1)) # dist_B[i] is the distance of the ith row of B
#     prod_dist = np.outer(dist_A, dist_B) # prod_dist[i,j] return the product of the distance of the ith row in A with the distance of the jth row in B
    return np.divide(prod, np.outer(dist_A, dist_B)) # returns the cosine similarity betwenen the two matrices, output[i,j] is the cosine similarity between the ith row in A and jth row in B.

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
def rank(query_idx, p, c, M, w1, w2, w3):
    """
    Return a list of reports indexes sorted by similarity of the bug reports (candidates) and new bug report (query)
    Cosine similarity is used to compare bug reports. 
    
    query_idx: query indexes
    p: product values of all bug reports (list)
    c: component values of all bug reports  (list)
    M: textual data representation of all bug reports  (Matrix)
    
    w1: parameter that controls the impact of the product
    w2: parameter  that controls the impact of the component
    w3: parameter  that controls the impact of textual similarity
    
    return: ranked list of indexes. 
    """
    if w3 == 0:
        cos_query = 0
    else:
        cos_query = cosine_similarity(M,M[query_idx:query_idx+1]) # It returns a numpy array of a (len(p),1) shape of similarities between the query and other bug reports.
    
    if w1 == 0:
        Fp = 0
    else:
        Fp = np.array([int(p[query_idx]==j) for j in p]).reshape(len(p),1) # It returns a numpy array of a (len(p),1) shape, of comparison between query product and other bug reports products.
        
    if w2 == 0:
        Fc = 0
    else:
        Fc = np.array([int(c[query_idx]==j) for j in c]).reshape(len(c),1) # It returns a numpy array of a (len(c),1) shape, of comparison between query component and other bug reports components.
    
    sim_query = w1*Fp + w2*Fc + w3*cos_query
    sim_query_arr = list(sim_query[:,0])
    
    sorted_indexes = list(list(zip(*sorted([(val, i) for i, val in enumerate(sim_query_arr)], reverse = True)))[1]) # It returns the list of indexes sorted
    sorted_indexes.remove(query_idx)
    
    return sorted_indexes
    

In [37]:
%%time
nix = rank(0, pro, comp, K, 0.5, 0.5, 1)
print(nix)
print(type(nix))

[153, 7824, 1181, 547, 7186, 6881, 4913, 3665, 8039, 1393, 372, 347, 3862, 7326, 7813, 399, 7076, 487, 611, 2379, 2312, 344, 2852, 4962, 1529, 2042, 380, 2853, 3796, 1280, 7098, 7931, 9474, 8225, 6903, 1349, 2346, 667, 6936, 444, 4739, 7392, 6826, 6914, 2072, 9377, 2491, 1360, 2887, 8355, 1305, 6925, 1382, 7488, 3632, 1992, 778, 8593, 7065, 700, 9303, 8834, 7770, 7413, 7381, 317, 7575, 1436, 9965, 3251, 7402, 602, 4136, 1759, 1682, 2267, 3928, 2266, 9410, 1404, 987, 450, 1671, 4717, 1748, 4201, 1346, 2876, 4125, 5146, 1760, 9248, 8582, 2762, 406, 407, 6444, 2215, 131, 2933, 305, 2624, 360, 3922, 1332, 1308, 7114, 193, 4182, 294, 3775, 186, 5258, 3411, 1451, 2632, 6217, 3976, 9668, 4997, 2772, 3022, 4332, 3033, 1581, 5777, 395, 1411, 1931, 1780, 1712, 1613, 9032, 2736, 629, 1204, 2302, 1162, 5531, 7493, 3664, 7515, 8203, 5684, 1330, 1889, 3354, 3653, 5836, 1073, 1533, 1157, 5661, 5605, 946, 577, 4042, 7502, 835, 498, 1005, 5725, 7477, 2111, 944, 316, 8255, 2335, 1240, 838, 2297, 4060, 2

## 7.3 - Évaluation / Evaluation

Vous allez tester différentes configurations du système de recommandations. Ces configurations seront comparées avec la [mean average precision (MAP) metric](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision). Plus les discussions pertinentes sont recommandées rapidement (c.-à-d. en haut de la liste), plus élevé sera le score MAP. Ressources supplémentaires pour comprendre MAP: [recall and precision over ranks](https://youtu.be/H7oAofuZjjE) et [MAP](https://youtu.be/pM6DJ0ZZee0).


La fonction *eval* évalue une configuration spécifique du système

---

We will test different configurations of our recommender system. These configurations are compared using the [mean average precision (MAP) metric](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision). Basically, the closer relevant threads are from ranked list begining, the higher MAP is. Additional materials to undertand MAP: [recall and precision over ranks](https://youtu.be/H7oAofuZjjE) and [MAP](https://youtu.be/pM6DJ0ZZee0).


The function *eval* evaluates a specific configurantion of our system



In [38]:
from statistics import mean 


def calculate_map(x):
    res = 0.0
    n = 0.0
    
    for query_id, corrects, candidate_ids in x:
        precisions = []
        for k, candidate_id in enumerate(candidate_ids):
            
            if candidate_id in corrects:
                prec_at_k = (len(precisions) + 1)/(k+1)
                precisions.append(prec_at_k)
                
            if len(precisions) == len(corrects):
                break
                            
        res += mean(precisions)
        n += 1
    
    return res/n
            

def eval(tokenization_type, vectorizer, enable_stop_words, enable_stemming, w1, w2, w3):
    reports = [r for r in report_index.values()]
    report_ids = [r["report_id"] for r in report_index.values()]
    prod_v, comp_v, M = nlp_pipeline(reports, tokenization_type, vectorizer, enable_stop_words, enable_stemming)
    report2idx = dict([(r['report_id'], idx) for idx,r in enumerate(reports)])
    rank_lists = []
    for query_id, corrects in test:
        query_idx =  report_ids.index(query_id)
        candidate_idxs = rank(query_idx, prod_v, comp_v, M, w1, w2, w3)
        candidate_ids = [ report_ids[idx] for idx in candidate_idxs]
                
        rank_lists.append((query_id, corrects, candidate_ids))
        
        
    return calculate_map(rank_lists)
    

## 7.4 - Question 9 (4 points)

Évaluez votre système avec chacune des configurations suivantes :

1. count(BoW) + space_tokenization
2. count(BoW) + nltk_tokenization
2. count(BoW) + space_punk_tokenization
3. count(BoW) + space_punk_tokenization + Stop words removal
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming
5. tf_idf + space_punk_tokenization
6. tf_idf + space_punk_tokenization + Stop words removal
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming 

**Pour chaque configuration :** 

1. Rapportez la performance avec $w_1=0$ et $w_2=0$

2. Réglez les paramètres $w_1$, $w_2$ et $w_3$ et rapportez les valeurs qui donne les 3 meilleurs et les 3 pires résultats.

**En plus, décrivez et comparez vos résultats. Répondez aux questions suivantes :**

- Quelle méthode de tokenization donne les meilleurs résultats ? À votre avis, pourquoi ?
- Les étapes de prétraitement ont-elles un impact positif ou négatif sur notre système ?
- Est-ce que TF-IDF permet d'obtenir de meilleures performances que CountBoW? Si oui, à votre avis, pourquoi ?
- Est-ce que la comparaison du composant et du produit affecte positivement notre méthode ? 

**Notez qu'il y a une valeur minimum de  MAP à atteindre pour chaque configuration en dessous de laquelle la question sera pénalisée de 50%.**

1. count(BoW) + space_tokenization: 0.090 
2. count(BoW) + nltk_tokenization: 0.090
2. count(BoW) + space_punk_tokenization: 0.120
3. count(BoW) + space_punk_tokenization + Stop words removal: 0.170
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming: 0.195
5. tf_idf + space_punk_tokenization: 0.210
6. tf_idf + space_punk_tokenization + Stop words removal: 0.210
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming: 0.215

---

Evaluate the system using each one of the following configurations:

1. count(BoW) + space_tokenization
2. count(BoW) + nltk_tokenization
2. count(BoW) + space_punk_tokenization
3. count(BoW) + space_punk_tokenization + Stop words removal
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming
5. tf_idf + space_punk_tokenization
6. tf_idf + space_punk_tokenization + Stop words removal
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming 

**For each configuration:** 

1. Report the method performance achieved when $w_1=0$ and $w_2=0$

2. Tune the parameters $w_1$, $w_2$ and $w_3$ and report the parameter values that achieve the 3 best and 3 worst results.

**Also, describe and compare the results found by you and answer the following questions:**

- Which tokenization strategy has achieved the best result? Why do you think this has occurred?
- Was our system negatively or positively impacted by data preprocessing steps?
- TF-IDF has achieved a better performance than CountBoW? If yes, why do you think that this has occurred?
- Did the component and product comparison positively affect our method? 

**Note that there is a minimum MAP value to achieve for each configuration (see below) below which the question will be penalized by 50%.**

1. count(BoW) + space_tokenization: 0.090 
2. count(BoW) + nltk_tokenization: 0.090
2. count(BoW) + space_punk_tokenization: 0.120
3. count(BoW) + space_punk_tokenization + Stop words removal: 0.170
4. count(BoW) + space_punk_tokenization + Stop words removal + Stemming: 0.195
5. tf_idf + space_punk_tokenization: 0.210
6. tf_idf + space_punk_tokenization + Stop words removal: 0.210
7. tf_idf + space_punk_tokenization + Stop words removal + Stemming: 0.215

In [39]:
import pandas as pd
from tabulate import tabulate

In [40]:
W = [(0,0,1), (1,0,0), (0,1,0), (0.33, 0.33, 0.33), (0.25, 0.25, 0.5), (0.1, 0.1, 0.8), (0.05, 0.05, 0.9), (0.025, 0.025, 0.95), (0.0125, 0.0125, 0.975), (0.01, 0.01, 0.98)]

In [41]:
def W_eval_dataframe(tokenization_type, vectorizer, enable_stop_words, enable_stemming, W):
    
    eval_dict = {
        'w1':[w1 for w1,w2,w3 in W],
        'w2':[w2 for w1,w2,w3 in W],
        'w3':[w3 for w1,w2,w3 in W],
        'MAP':[eval(tokenization_type, vectorizer, enable_stop_words, enable_stemming, w1, w2, w3) for (w1,w2,w3) in tqdm(W)]
    }
    
    W_eval_df = pd.DataFrame(eval_dict, index= [f'Combination {i}' for i in range(1,len(W)+1)])
    
    print(f"---> The performance of {len(W)} different combinations of (w1,w2,w3):")
    print(tabulate(W_eval_df, headers = 'keys', tablefmt = 'psql'))
    print('\n')
    
    three_best = W_eval_df.sort_values(by = ['MAP'], ascending = False)[:3]
    three_best.index = [1,2,3]
    three_worst = W_eval_df.sort_values(by = ['MAP'])[:3]
    three_worst.index = [1,2,3]
    
    print('---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:')
    print(tabulate(three_best, headers = 'keys', tablefmt = 'psql'))
    print('\n')
    print('---> The parameters (w1,w2,w3) that achieve the 3 worst MAP values are:')
    print(tabulate(three_worst, headers = 'keys', tablefmt = 'psql'))
    print('\n')
    

### 1. count(BoW) + space_tokenization


In [42]:
%%time
W_eval_dataframe(tokenize_space, transform_count_bow, False, False, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:03<00:00, 12.32s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.0838898  |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.0881391  |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.0877804  |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.095196   |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.0903597  |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.087135   |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.0874612  |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.0873582  |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+------+------+------+-----------+
|    |   w

### 2. count(BoW) + nltk_tokenization

In [43]:
%%time
W_eval_dataframe(tokenize_nltk, transform_count_bow, False, False, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:01<00:00, 24.18s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.0820239  |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.0894025  |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.089545   |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.0956418  |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.0970113  |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.0886784  |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.0870183  |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.0859703  |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+------+------+------+-----------+
|    |   w

### 3. count(BoW) + space_punk_tokenization


In [44]:
%%time
W_eval_dataframe(tokenize_space_punk, transform_count_bow, False, False, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:17<00:00, 13.70s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.119054   |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.119447   |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.119205   |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.129894   |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.12997    |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.126752   |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.121513   |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.121328   |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+-------+-------+------+----------+
|    |   

### 4. count(BoW) + space_punk_tokenization + Stop words removal 


In [45]:
%%time
W_eval_dataframe(tokenize_space_punk, transform_count_bow, True, False, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:58<00:00, 11.81s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.167225   |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.143961   |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.143827   |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.171352   |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.178748   |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.173612   |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.169366   |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.16848    |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+-------+-------+------+----------+
|    |   

### 5. count(BoW) + space_punk_tokenization + Stop words removal + Stemming 

In [46]:
%%time
W_eval_dataframe(tokenize_space_punk, transform_count_bow, True, True, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:03<00:00, 24.34s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.194213   |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.155522   |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.155455   |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.194238   |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.200108   |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.196189   |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.199024   |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.199868   |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+--------+--------+-------+----------+
|    |

### 6. tf_idf + space_punk_tokenization 

In [47]:
%%time
W_eval_dataframe(tokenize_space_punk, transform_tf_idf_bow, False, False, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:11<00:00, 13.13s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.201861   |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.158464   |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.158331   |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.204455   |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.214397   |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.210077   |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.206889   |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.205866   |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+--------+--------+-------+----------+
|    |

### 7.  tf_idf + space_punk_tokenization + Stop words removal


In [48]:
%%time
W_eval_dataframe(tokenize_space_punk, transform_tf_idf_bow, True, False, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:59<00:00, 11.91s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.19926    |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.158668   |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.158557   |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.203172   |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.212298   |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.205445   |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.204717   |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.203802   |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+--------+--------+-------+----------+
|    |

### 8.tf_idf + space_punk_tokenization + Stop words removal + Stemming 

In [49]:
%%time
W_eval_dataframe(tokenize_space_punk, transform_tf_idf_bow, True, True, W)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:16<00:00, 25.61s/it]

---> The performance of 10 different combinations of (w1,w2,w3):
+----------------+--------+--------+-------+------------+
|                |     w1 |     w2 |    w3 |        MAP |
|----------------+--------+--------+-------+------------|
| Combination 1  | 0      | 0      | 1     | 0.212493   |
| Combination 2  | 1      | 0      | 0     | 0.00391105 |
| Combination 3  | 0      | 1      | 0     | 0.0136751  |
| Combination 4  | 0.33   | 0.33   | 0.33  | 0.165059   |
| Combination 5  | 0.25   | 0.25   | 0.5   | 0.16522    |
| Combination 6  | 0.1    | 0.1    | 0.8   | 0.217289   |
| Combination 7  | 0.05   | 0.05   | 0.9   | 0.224494   |
| Combination 8  | 0.025  | 0.025  | 0.95  | 0.222491   |
| Combination 9  | 0.0125 | 0.0125 | 0.975 | 0.216999   |
| Combination 10 | 0.01   | 0.01   | 0.98  | 0.215698   |
+----------------+--------+--------+-------+------------+


---> The parameters (w1,w2,w3) that achieve the 3 best MAP values are:
+----+-------+-------+------+----------+
|    |   

### Comparison of the results:

#### 1.  The tokenization strategy:
The tokenization strategy has achieved the best result is space_punk_tokenization since this strategy eliminates punctuations, knowing that punctuations tends to occur in the bug reports description, including punctuation when calculating the similarity is a bad idea, since it doesn't give any insight about similarity, so it only alters the ranking of similar reports to the query, that's why space_punk_tokenization achieved the best results in all of the configurations, this knowing that the two other tokenizations strategies keep the punctuations as tokens.

#### 2.  The impact of data preprocessing steps on our system:
We notice that our system has been impacted positively with our data preprocessing steps and this for the two vectorizers: count_bow and tf_idf and this for most of the combinations used on the configurations, each time we rach higher MAP values (by removing stop words, and we reach higher MAP values with removing stop words and enable stemming). 

#### 3.  TF-IDF vs Count-BOW:
TF-IDF showed better results than count_bow vectorization for the same parameters combinations and the same data preprocessing steps, because TF-IDF penalizes the frequent words that are in most of documents, because these words don't discriminate documents or gives an insight on the similarity of documents, but in the other hand this strategy rewards the rarer words, that's why we reach better results. 

#### 4.  The impact of the component and product comparison on our system:
The product and component comparison defenitely affect positively our system, We notice that we get the higher MAP values for any type of vectorization, and data processing steps non values for w1 and w2 compared to when w1=0 and w2=0. 